# Agentic RAG

**실습자료**
- 제목 : AI.GOV 해외동향 2025-1호
- 링크 : [한국지능사회정보진흥원](https://www.innovation.go.kr/ucms/bbs/B0000051/view.do?nttId=17774&menuNo=300145&pageIndex=)  
- 출처 : NIA 한국지증정보원원

*링크에서 파일을 다운로드 받은후 최상단 data 폴더에 위치시켜주세요*

In [1]:
# RAG 구성을 위한 패키지 설치치
%pip install pypdf faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

OPENAI_API_KEY가 설정되어 있나요?: sk-proj-qg...


#### RAG 구성

**Document load**

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/[AI.GOV_해외동향]_2025-1호.pdf")

docs = loader.load()

**TextSplite**

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],  # 문단 → 줄 → 문장 → 단어 순
    chunk_size=1000,    
    chunk_overlap=100
)

chunks = recursive_splitter.split_documents(docs)

print(f"총 {len(chunks)}개 청크 생성\n")

for i, chunk in enumerate(chunks, 1):
    chunk_length = len(chunk.page_content)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk.page_content}'")

총 33개 청크 생성

청크 1 (142자):
'1
해외 에이전트 기술 및 서비스 개발 동향AI 
을 보완하는 기술 현황LLM RAG 2.0 
정부의 적극적 활용 정책과 반향 미국 영국 사례AI : ·
주요국 인프라 투자 경쟁 가속화AI 
최신 보고서 의 미래 예견적 거버넌스 전략[OECD ] AI :'
청크 2 (158자):
'발행처 한국지능정보사회진흥원: ■ 
발행인 황종성: ■ 
작성자 한국지능정보사회진흥원 디지털플랫폼정부본부 정책기획팀: ■ 
장우림 책임 조원선 선임         - (woolim@nia.or.kr), (wscho@nia.or.kr)
보고서 온라인 서비스: www.nia.or.kr■'
청크 3 (165자):
'01.해외 에이전트 기술 및 서비스 개발 동향AI 4
02. 을 보완하는 기술 현황LLM RAG 2.0 9
03.정부의 적극적 활용 정책과 반향AI : 미국 영국 사례· 12
04.주요국 인프라 투자 경쟁 가속화 AI 15
05. 최신 보고서[OECD ]의 미래 예견적 거버넌스 전략AI : 17'
청크 4 (985자):
'4
01
해외 에이전트 기술 및 서비스 개발 동향AI 
Reading Point
  ￭ 에이전트가 인공지능 산업에서 AI 년의 가장 중요한 트렌드2025 로 거론되고 있으며, 
‘Computer Use’, ‘Operator’, ‘Manus AI’등 다양한 에이전트 서비스가 잇따라 출시 AI 
  ￭현재 초기 단계인 에이전트는 빠른 속도로 발전하고 있으며 에이전트의 AI , AI 
대중화는 향후 노동시장 및 산업 전반에 큰 파급 효과를 미칠 것으로 전망
기술 및 서비스 개발의 주요 트렌드로 부상한 에이전트AI AI Ⅰ
  ￭ 에이전트AI : 워크플로를 설계하고 사용 가능한 도구를 활용하여 사용자 또는 다른 
시스템을 대신하여 작업을 자율적으로수행할 수 있는 시스템이나 프로그램 1)
 ▸ 에이전트 또는 가 분야의 년 주요 트렌드로 부상AI (AI Agent Agentic AI) AI 2025
   - 가트너 분야 전

**Embedding**

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

**VectorStore**

In [6]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=chunks, embedding=embeddings)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************XxwA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

**Retriever 생성**

In [7]:
# 2. MMR Search  
mmr_retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,  # 검색 문서 수를 3개로 설정
        "lambda_mult": 0.8,  # 다양성 중시
    }
)

Retriever 사용해보기

In [8]:
query = "중국, 미국의 AI 동향에 대해 알려줘"

print(f"질문 : {query}")
mmr_results = mmr_retriever.invoke(query)
print("=== MMR 검색 (k=3) ===")
for i, doc in enumerate(mmr_results):
    print(f"{i+1}. {doc.page_content}")

질문 : 중국, 미국의 AI 동향에 대해 알려줘
=== MMR 검색 (k=3) ===
1. 1
해외 에이전트 기술 및 서비스 개발 동향AI 
을 보완하는 기술 현황LLM RAG 2.0 
정부의 적극적 활용 정책과 반향 미국 영국 사례AI : ·
주요국 인프라 투자 경쟁 가속화AI 
최신 보고서 의 미래 예견적 거버넌스 전략[OECD ] AI :
2. 중국 스타트업 의 등 에이전트 서비스 잇따라 출시Butterfly Effect ‘Manus AI’ AI 
   - 다만 아직 주요 기능은 실험적이거나 초기 베타 버전으로 제공되고 있음, ( ) 
1) IBM. (2024, January 3) What are AI agents? https://www.ibm.com/think/topics/ai-agents
2) Gartner. (2024. October 21) Top 10 Strategic Technology Trends for 2025.
https://www.gartner.com/en/articles/top-technology-trends-2025
3) Fortune. (2025, January 7) Nvidia’s Jensen Huang says AI agents are ‘a multi-trillion-dollar opportunity’ and 
‘the age of AI Agentics is here’. https://finance.yahoo.com/news/nvidia-jensen-huang-says-ai-044815659.html
3. 14
공공부문 활용에 관한 시사점AI Ⅳ
 ▸전문가들은 가 정부 업무를 로 대체하려는 계획을 추진하는 것에 대해 DOGE AI
충분한 검증과 공적 논의 없이 성급히 도입하는 것은 위험하다는 의견 표명18)
   - 알렉스 한나 연구소장은 의 투명성 부족 문제를 지적하며 책임 소재의 모호성을 DAIR AI
비판하였으며 프랭크 파스콸레 브루클린 법대 교수는 정부 정책 결정에 를 무분별, AI
하게 사용하는 것은 공적 책임성을 훼손한

구축한 `retriever` 를 Agent가 사용할 수 있는 도구로 변환

In [9]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=mmr_retriever,
    name="pdf_search",
    description="PDF 문서에서 검색 결과를 반환합니다."
)

# Tavily API를 사용해 검색 Agent 구현

이제 실시간 정보 검색이 가능한 Agent를 만들어보겠습니다.

### 검색 도구 설정

**주의**: Tavily 검색 도구를 사용하려면 API 키가 필요합니다. 
- 발급 주소: https://app.tavily.com/
- `.env` 파일에 `TAVILY_API_KEY=your_key_here` 추가


In [10]:
# Tavily 검색 도구 설정 (API 키가 필요합니다)
from langchain_community.tools.tavily_search import TavilySearchResults

# 검색 도구 생성
web_search_tool = TavilySearchResults(
    max_results=3,
    include_answer=True,
    include_raw_content=False,
    search_depth="advanced"
)

C:\Users\jtw57\AppData\Local\Temp\ipykernel_16484\22243656.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(


Agent가 사용할 도구 목록 정의

In [11]:
tools = [retriever_tool, web_search_tool]

#### Agent 생성

프롬프트 및 llm 객체 생성

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model

# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are very powerful assistant. 
            정보를 검색할때는 반드시 `pdf_search` 도구를 사용하세요.
            만약 PDF 문서에서 찾을 수 없다면, `tavily_search` 도구를 사용해 웹에서 검색 하세요."""
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# LLM 정의의
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

모델 생성

In [13]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

AgentExecutor 생성

In [14]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Agent Executor 생성
agetnt_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # 실행 과정을 자세히 보여줍니다
    handle_parsing_errors=True  # 파싱 에러를 자동으로 처리합니다
)

In [15]:
result = agetnt_executor.invoke({"input": "중국의 AI 동향에 pdf 기반으로 설명해줘"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '중국 AI 동향'}`


1
해외 에이전트 기술 및 서비스 개발 동향AI 
을 보완하는 기술 현황LLM RAG 2.0 
정부의 적극적 활용 정책과 반향 미국 영국 사례AI : ·
주요국 인프라 투자 경쟁 가속화AI 
최신 보고서 의 미래 예견적 거버넌스 전략[OECD ] AI :

중국 스타트업 의 등 에이전트 서비스 잇따라 출시Butterfly Effect ‘Manus AI’ AI 
   - 다만 아직 주요 기능은 실험적이거나 초기 베타 버전으로 제공되고 있음, ( ) 
1) IBM. (2024, January 3) What are AI agents? https://www.ibm.com/think/topics/ai-agents
2) Gartner. (2024. October 21) Top 10 Strategic Technology Trends for 2025.
https://www.gartner.com/en/articles/top-technology-trends-2025
3) Fortune. (2025, January 7) Nvidia’s Jensen Huang says AI agents are ‘a multi-trillion-dollar opportunity’ and 
‘the age of AI Agentics is here’. https://finance.yahoo.com/news/nvidia-jensen-huang-says-ai-044815659.html

해외동향 호AI.GOV 2025-1 7
주요 에이전트 서비스 AI Manus AI (Butterfly Effect, '25.3.)③ Ⅴ
 ▸ 는 중국의 스타트업 가 개발한 자율형 인공지능 에이전트로Manus Butterfly Effect , 
인간의 개입 없이 복잡한 작업을 독립적으로 수행할 수 있도록 설계
   - 는 가 주식 분석부터 부동산 추천까지 다양

In [16]:
response = agetnt_executor.invoke({"input": "2025년 한국 프로야구 진출 예상팀 명단을 알려줘"})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2025년 한국 프로야구 진출 예상팀 명단'}`


[{'title': 'LG, 2025시즌 포스트시즌 진출 확정!→7시즌 연속으로 두산과 어깨 ...', 'url': 'https://sports.news.nate.com/view/20250908n18418', 'content': '| 순위 | 승 | 무 | 패 | 승률 | 게임차 |\n| 1LG | 78 | 3 | 47 | 0.624 | 0.0 |\n| 2한화 | 73 | 3 | 52 | 0.584 | 5.0 |\n| 3SSG | 65 | 4 | 58 | 0.528 | 12.0 |\n| 4삼성 | 65 | 2 | 62 | 0.512 | 14.0 |\n| 5KT | 63 | 4 | 62 | 0.504 | 15.0 |\n| 6롯데 | 62 | 6 | 62 | 0.500 | 15.5 |\n| 7NC | 58 | 6 | 62 | 0.483 | 17.5 |\n| 8KIA | 58 | 4 | 64 | 0.475 | 18.5 |\n| 9두산 | 56 | 6 | 65 | 0.463 | 20.0 |\n| 10키움 | 41 | 4 | 85 | 0.325 | 37.5 |\n| 2025.09.09 기준 | | | | | |\n\n \n\n### 생생화보 [...] 공감수\n   54](//sports.news.nate.com/view/20250909n00098)\n2. 7\n   \'72억 집\' 손연재, 명품 가방 \'내돈내산\'에도 남편 눈치 "비밀로 하고 몰래 사"(손연재)\n\n   공감수\n   47\n3. 8\n   \'뺨 터치 논란\' KT 장성우 "투-포수, 신뢰 가장 중요…주변 의식 안 해"\n\n   공감수\n   30\n4. 9\n   제주 복싱대회서 쓰러진 중학생 선수 엿새째 의식불명\n\n   공감수\n   28\n5. 10\n   [[